Goal: Preprocess the WCR information further based on well age, depth and location to prepare a general dataset useful for the Cosumnes modeling work. This was originally in the main model code, but was set aside to reduce clutter.

In [1]:
import os
from os.path import basename, dirname, exists, join
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
doc_dir = os.getcwd()
while basename(doc_dir) != 'Documents':
    doc_dir = dirname(doc_dir)
# dir of all gwfm data
gwfm_dir = dirname(doc_dir)+'/Box/research_cosumnes/GWFlowModel'
gwfm_dir

'C:\\Users\\ajcalder/Box/research_cosumnes/GWFlowModel'

In [3]:
# model can't run before 2010 due to input limitations for crop coefficients so this will be considered the oldest date
# for wells (going 60 years back from 2010 will allow a 70 year age)
strt_date = pd.to_datetime('2010-10-01')


In [4]:
# Load model grid as geopandas object
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')

In [5]:
nrow = grid_p.row.max()
ncol = grid_p.column.max()

In [13]:
wells = pd.read_csv(gwfm_dir+'/WEL_data/all_wells_type.csv')
wells_grid = gpd.GeoDataFrame(wells, geometry = gpd.points_from_xy(wells.easting,wells.northing), crs = 'epsg:32610')
# remove wells that didn't exist before model start date, could improve later to start pumping mid-model
wells_grid.DateWorkEnded = pd.to_datetime(wells_grid.DateWorkEnded )
wells_grid['well_age_days'] = (strt_date - wells_grid.DateWorkEnded).dt.days

# if wells_grid.row.min()==1:
wells_grid.row = (wells_grid.row-1).astype(int)
wells_grid.column = (wells_grid.column -1).astype(int)

In [14]:
wells_grid['depth_m'] = wells_grid.TotalCompletedDepth*0.3048
wells_grid['flux'] = 0
wells_grid['layer'] = 0

In [30]:
wells_grid.to_csv(gwfm_dir+'/WEL_data/wells_grid.csv', index=False)

In [31]:
# wells_grid = pd.read_csv(gwfm_dir+'/WEL_data/wells_grid.csv')
# wells_grid = gpd.GeoDataFrame(wells_grid, geometry = gpd.points_from_xy(wells_grid.easting, wells_grid.northing),
#                               crs='epsg:32610')

In [8]:
wells_grid_ag = wells_grid.copy()
wells_grid_ag = wells_grid.loc[wells_grid.Simple_type == 'irrigation'].dissolve('node', aggfunc = 'mean')
wells_grid_ag.geometry = wells_grid.loc[wells_grid.Simple_type == 'irrigation'].dissolve('node', aggfunc = 'first').geometry

# remove wells older than 60 years for ag, think of Teichert wells from 60s (domestic wells age out earlier)
print(wells_grid_ag.well_age_days.notna().sum(), wells_grid_ag.shape, (wells_grid_ag.well_age_days < 60*365).sum())
wells_grid_ag = wells_grid_ag[wells_grid_ag.well_age_days < 60*365]

# the longest road on Teichert is 1500 m, and on a regional scale these seem to be the largest ag fields.
# Visually a 500 m buffer results in a small amount of ag, 1000 m causes near total coverage in upper basin
ag_wells_buffer = wells_grid_ag.loc[:,['well_age_days','geometry']]
ag_wells_buffer.geometry = ag_wells_buffer.geometry.buffer(1500)
pasture_ag_land = gpd.sjoin(grid_p, ag_wells_buffer, op='within')

C:\Users\ajcalder\anaconda3\envs\geo_env\Lib\site-packages\geopandas\geodataframe.py:1676: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = data.groupby(**groupby_kwargs).agg(aggfunc)
C:\Users\ajcalder\anaconda3\envs\geo_env\Lib\site-packages\IPython\core\interactiveshell.py:3382: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


280 (280, 17) 267


In [9]:
# wells_grid_ag is the same thing, but already dissolved by node
ag_well = wells_grid.loc[wells_grid.Simple_type == 'irrigation']
mean_ag_well_depth = ag_well.TotalCompletedDepth.mean()*0.3048
print('Mean ag well_depth ',mean_ag_well_depth)

Mean ag well_depth  107.05973


In [10]:
ncell = nrow*ncol

df = ag_well.depth_m
# Get the xy cell centers for the model grid
# xy = np.append(m.modelgrid.xcellcenters,m.modelgrid.ycellcenters, axis = 0)
# reshape the xy data
# out_xy = np.transpose(np.vstack([np.reshape(xy[0:nrow], ncell),np.reshape(xy[nrow:],ncell)]))

out_xy = np.transpose(np.vstack([grid_p.geometry.centroid.x.values, grid_p.geometry.centroid.y.values]))


from scipy.interpolate import griddata
in_xy = np.transpose(np.vstack([ag_well.geometry.x.values, ag_well.geometry.y.values]))
# Final reshaped array of interpolated ET
ag_well_depth_arr = np.zeros((nrow, ncol))

# scipy interpolation based on point locations
notna_data = df.notna().values
grid = griddata(in_xy[notna_data], df.loc[notna_data].values, xi = out_xy, method = 'nearest')

ag_well_depth_arr[:,:] = np.reshape(grid, (nrow,ncol))

# plt.imshow(ag_well_depth_arr)
# plt.colorbar()

In [30]:
np.savetxt('ag_well_depth_arr.tsv',ag_well_depth_arr, delimiter='\t')